In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
import numpy as np

EPOCHS = 5
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10

N_HIDDEN = 100
VALIDATION_SPLIT = 0.5
IMG_ROWS, IMG_COLS = 28, 28

INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)

DROPOUT = 0.3

mnist = keras.datasets.mnist 
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train.reshape(60000, IMG_ROWS, IMG_COLS, 1)
X_test = X_test.reshape(10000, IMG_ROWS, IMG_COLS, 1)

X_train, X_test = X_train / 255.0, X_test / 255.0

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

print('complete')

complete


In [2]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 128, 512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'RMSprop']))

METRIC_ACCURACY = 'accuracy'
METRIC_LOSS = 'loss'

def create_model(hidden_layers, input_shape, dropout, output_classes, opt, metrics):
    tf.random.set_seed(1234)

    model = tf.keras.models.Sequential()
    model.add(keras.layers.Convolution2D(20, (5, 5), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(keras.layers.Convolution2D(50, (5, 5), activation='relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(hidden_layers, name='hidden_dense_layer',activation='relu'))
    model.add(keras.layers.Dense(output_classes, name='output_dense_layer',activation='softmax'))
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=metrics)
    
    return model

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy'),
            hp.Metric(METRIC_LOSS, display_name='Loss')],
    )
    

def train_test_model(hparams):
    model = create_model(hparams[HP_NUM_UNITS], INPUT_SHAPE, hparams[HP_DROPOUT], NB_CLASSES, hparams[HP_OPTIMIZER], ['accuracy'])

    model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT) # Run with 1 epoch to speed things up for demo purposes
    loss, accuracy = model.evaluate(np.array(X_test), np.array(Y_test))
    del model
    return loss, accuracy

def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss, accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_LOSS, loss, step=1)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
print('complete')

complete


In [ ]:

session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'RMSprop'}
Train on 30000 samples, validate on 30000 samples
Epoch 1/5
30000/30000 [==============================] - 20s 677us/sample - loss: 0.4550 - accuracy: 0.8676 - val_loss: 0.1623 - val_accuracy: 0.9503
Epoch 2/5
30000/30000 [==============================] - 16s 523us/sample - loss: 0.1245 - accuracy: 0.9622 - val_loss: 0.1348 - val_accuracy: 0.9577
Epoch 3/5
30000/30000 [==============================] - 16s 523us/sample - loss: 0.0849 - accuracy: 0.9742 - val_loss: 0.0755 - val_accuracy: 0.9765
Epoch 4/5
30000/30000 [==============================] - 17s 554us/sample - loss: 0.0637 - accuracy: 0.9800 - val_loss: 0.0672 - val_accuracy: 0.9801
Epoch 5/5
10000/10000 [==============================] - 3s 273us/sample - loss: 0.0470 - accuracy: 0.9860
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Train on 30000 samples, validate on 30000 samples
Epoch 1/5
30000/30000 [=====

30000/30000 [==============================] - 15s 515us/sample - loss: 0.5041 - accuracy: 0.8541 - val_loss: 0.3405 - val_accuracy: 0.9021
Epoch 3/5
30000/30000 [==============================] - 15s 509us/sample - loss: 0.3281 - accuracy: 0.9040 - val_loss: 0.2664 - val_accuracy: 0.9225
Epoch 4/5
30000/30000 [==============================] - 17s 557us/sample - loss: 0.2601 - accuracy: 0.9242 - val_loss: 0.2149 - val_accuracy: 0.9378
Epoch 5/5
10000/10000 [==============================] - 3s 253us/sample - loss: 0.1736 - accuracy: 0.9471
--- Starting trial: run-9
{'num_units': 128, 'dropout': 0.3, 'optimizer': 'RMSprop'}
Train on 30000 samples, validate on 30000 samples
Epoch 1/5
30000/30000 [==============================] - 23s 776us/sample - loss: 0.3220 - accuracy: 0.8997 - val_loss: 0.1003 - val_accuracy: 0.9699
Epoch 2/5
30000/30000 [==============================] - 18s 611us/sample - loss: 0.0902 - accuracy: 0.9718 - val_loss: 0.0807 - val_accuracy: 0.9751
Epoch 3/5
30000/30